<a href="https://colab.research.google.com/github/tchintchie/stocks/blob/master/stockprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
df = web.get_data_yahoo("TSLA")

In [3]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-07-07,275.200012,260.769989,275.000000,267.880005,6105100,267.880005
2015-07-08,260.799988,254.309998,259.320007,254.960007,6221100,254.960007
2015-07-09,262.950012,256.790009,259.079987,257.920013,3334100,257.920013
2015-07-10,263.000000,257.820007,262.220001,259.149994,2610900,259.149994
2015-07-13,262.549988,256.049988,262.250000,262.160004,2960300,262.160004


In [5]:
df["hl_pct"] = (df.High - df.Low)/df.Low*100

In [7]:
df["pct_change"] = (df["Adj Close"] - df.Open)/df.Open*100

In [8]:
df = df[["Adj Close","hl_pct", "pct_change", "Volume"]]
df.head()

,Adj Close,hl_pct,pct_change,Volume
Date,,,,
2015-07-07,267.880005,5.533621,-2.589089,6105100
2015-07-08,254.960007,2.552000,-1.681321,6221100
2015-07-09,257.920013,2.398849,-0.447728,3334100
2015-07-10,259.149994,2.009151,-1.170775,2610900
2015-07-13,262.160004,2.538567,-0.034317,2960300


In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVR
import math

In [13]:
forecast_col = "Adj Close"
df.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))

In [14]:
forecast_out

13

In [15]:
df["label"] = df[forecast_col].shift(-forecast_out)

In [17]:
df.head(15)

,Adj Close,hl_pct,pct_change,Volume,label
Date,,,,,
2015-07-07,267.880005,5.533621,-2.589089,6105100,265.410004
2015-07-08,254.960007,2.552000,-1.681321,6221100,253.009995
2015-07-09,257.920013,2.398849,-0.447728,3334100,264.820007
2015-07-10,259.149994,2.009151,-1.170775,2610900,263.820007
2015-07-13,262.160004,2.538567,-0.034317,2960300,266.790009
2015-07-14,265.649994,2.103559,1.354440,1907600,266.149994
2015-07-15,263.140015,2.064257,-1.349620,2021600,259.989990
2015-07-16,266.679993,1.535191,0.931039,1616000,266.279999
2015-07-17,274.660004,2.717617,0.792662,5004100,270.130005


In [18]:
df.dropna(inplace=True)

In [19]:
features = df.drop(["label"], axis = 1)
target = df.label

In [20]:
print("features: ", features.shape)
print("target: ", target.shape)

features:  (1245, 4)
target:  (1245,)


In [23]:
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [27]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size = 0.2)

In [29]:
print("train_features: ", train_features.shape)
print("test_features: ", test_features.shape)
print("train_target: ", train_target.shape)
print("test_target: ", test_target.shape)

train_features:  (996, 4)
test_features:  (249, 4)
train_target:  (996,)
test_target:  (249,)


In [32]:
model = SVR()
model.fit(train_features, train_target)
score = model.score(test_features, test_target)

In [33]:
score

0.18942414170645117

In [34]:
linreg = LinearRegression()
linreg.fit(train_features, train_target)
lin_score = linreg.score(test_features, test_target)

In [35]:
lin_score

0.8452615709466837

In [36]:
# testing different kernels for svr
for k in ["linear", "poly", "rbf","sigmoid"]:
  clf = SVR(kernel = k)
  clf.fit(train_features, train_target)
  score = clf.score(test_features, test_target)
  print(k, score)

linear 0.8248994063643849
poly 0.4826965878472556
rbf 0.18942414170645117
sigmoid 0.4969291756602978
